In [7]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from datasketch import MinHash, MinHashLSH

from time import time # needed to measure the elapsed time
from itertools import zip_longest
from collections import Counter

In [8]:
print("This is the answer to part 1 of the exercise\n")

train_ds_comma=pd.read_csv("train.csv")
# train_ds_line=pd.read_csv("train.csv",sep='|')
test_ds_comma=pd.read_csv("test_without_labels.csv")
# test_ds_line=pd.read_csv("test_without_labels.csv",sep='|')

fraction=0.1

# Split the dataset into a smaller training set and a smaller test set
train_subset, _ = train_test_split(
    train_ds_comma, 
    train_size=fraction, 
    stratify=train_ds_comma['Label'],  # Ensure stratified sampling
    random_state=42
)

train_subset.to_csv('train_subset.csv', index=False)


test_subset=test_ds_comma.sample(frac=fraction,random_state=42)
test_subset.to_csv('test_subset.csv',index=False)

train_subset=pd.read_csv('train_subset.csv')
test_subset=pd.read_csv('test_subset.csv')

This is the answer to part 1 of the exercise



In [ ]:

# vctrz=CountVectorizer()
vctrz= TfidfVectorizer()
# vctrz= HashingVectorizer()
x_train=vctrz.fit_transform(train_subset['Content'])
y_train=train_subset['Label']
x_test=vctrz.transform(test_subset['Content'])

classifiers={'Random Forest': RandomForestClassifier(),
            #  'SVM': SVC(),
             'SVM':LinearSVC()}
results={}

for name,clfr in classifiers.items():
    print("\nTesting:",name)
    scores=cross_val_score(clfr,x_train,y_train,cv=5,scoring='accuracy')
    results[name]=scores.mean()

print("5-Fold Cross-Validation Results:", results)




Testing: Random Forest

Testing: SVM
5-Fold Cross-Validation Results: {'Random Forest': np.float64(0.8736925558183557), 'SVM': np.float64(0.949996438190601)}


In [13]:
sorted_results=dict(sorted(results.items(), key=lambda item: item[1],reverse=True))
print(sorted_results)

best_fit_name= list(sorted_results.keys())[0]

best_fit_method=classifiers[best_fit_name]
best_fit_method.fit(x_train,y_train)

test_subset['Predicted'] = best_fit_method.predict(x_test)
test_subset[['Id', 'Predicted']].to_csv('testSet_categories.csv', index=False)

{'SVM': np.float64(0.949996438190601), 'Random Forest': np.float64(0.8736925558183557)}
